In [ ]:
#(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
#(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
#(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
#(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
#(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
#(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
#(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
#(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
#(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
#(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
#(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
#(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
#(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
#(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
#(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
#(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')
#(16, 86416, 'CIFTI_STRUCTURE_PALLIDUM_RIGHT')
#(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
#(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
#(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
#(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')

In [1]:
cd /mnt/c/Users/xavie/Dropbox\ \(MIT\)/MIT/_ACTIVE_PROJECTS/homotopy/projecting_to_cerebralcortex

/mnt/c/Users/xavie/Dropbox (MIT)/MIT/_ACTIVE_PROJECTS/homotopy/projecting_to_cerebralcortex


In [2]:
import os, glob, nibabel as nib, subprocess, numpy as np, matplotlib.pyplot as plt, pandas as pd, matplotlib as mpl, nilearn
import matplotlib.cm as cm, seaborn as sns, matplotlib
from nilearn import plotting
import subprocess, numpy as np, nibabel as nib
from sklearn.metrics import pairwise_distances
from PIL import Image
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn import metrics
import numpy as np

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [45]:
dcon = nib.load('/mnt/c/Users/xavie/Desktop/HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii').get_data()
caudategradients = nib.load('result_caudatebilateral.dscalar.nii').get_data()
cerebellumgradients = nib.load('result_fromcerebellumtocerebellum.dscalar.nii').get_data()
lenticulargradients = nib.load('result_pallidumandputamen.dscalar.nii').get_data()
thalamusgradients = nib.load('result_thalamus_bilateral.dscalar.nii').get_data()
cerebralcortexgradients = nib.load('result_fromcortextocortex_gradients_moregradients.dscalar.nii').get_data()

dcon_thalamus = dcon[88746:, :].copy()
thalamus_grad1 = thalamusgradients[0,0,0,0,0,:].copy()
thalamus_grad2 = thalamusgradients[0,0,0,0,1,:].copy()

dcon_lenticular = dcon[86119:88746, :].copy()
lenticular_grad1 = lenticulargradients[0,0,0,0,0,:].copy()
lenticular_grad2 = lenticulargradients[0,0,0,0,1,:].copy()

dcon_cerebellum = dcon[65289:83142, :].copy()
cerebellum_grad1 = cerebellumgradients[0,0,0,0,0,:].copy()
cerebellum_grad2 = cerebellumgradients[0,0,0,0,1,:].copy()
cerebellum_grad3 = cerebellumgradients[0,0,0,0,2,:].copy()

dcon_caudate = dcon[63806:65289, :].copy()
caudate_grad1 = caudategradients[0,0,0,0,0,:].copy()
caudate_grad2 = caudategradients[0,0,0,0,1,:].copy()

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


In [74]:
gradient_to_examine=caudate_grad2
dcon_to_examine=dcon_caudate
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('caudate_grad2_ProjectedToCerebralCortex.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [78]:
gradient_to_examine=cerebellum_grad3
dcon_to_examine=dcon_cerebellum
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('cerebellum_grad3_ProjectedToCerebralCortex.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [79]:
gradient_to_examine=thalamus_grad1
dcon_to_examine=dcon_thalamus
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('thalamus_grad1_ProjectedToCerebralCortex.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [80]:
gradient_to_examine=lenticular_grad1
dcon_to_examine=dcon_lenticular
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('lenticular_grad1_ProjectedToCerebralCortex.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [82]:
## the same with absolute values:

In [83]:
gradient_to_examine=np.absolute(caudate_grad2)
dcon_to_examine=dcon_caudate
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('caudate_grad2_ProjectedToCerebralCortex_AbsoluteValues.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [81]:
gradient_to_examine=np.absolute(cerebellum_grad3)
dcon_to_examine=dcon_cerebellum
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('cerebellum_grad3_ProjectedToCerebralCortex_AbsoluteValues.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [84]:
gradient_to_examine=np.absolute(thalamus_grad1)
dcon_to_examine=dcon_thalamus
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('thalamus_grad1_ProjectedToCerebralCortex_AbsoluteValues.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [85]:
gradient_to_examine=np.absolute(lenticular_grad1)
dcon_to_examine=dcon_lenticular
dict_temp = {}
i=0
for x in np.arange(len(gradient_to_examine)):
    dict_temp[i] = np.ndarray.flatten(dcon_to_examine[x]) * gradient_to_examine[x]
    i=i+1
    
weighted_conn = np.zeros((91282,))
for x in np.arange(len(gradient_to_examine)):
    weighted_conn = weighted_conn + np.array(dict_temp[x])
weighted_conn = weighted_conn[:59412] # to exclude everything except cerebral cortex
weighted_conn.shape = (59412,1)

#generate dscalar file
emb = weighted_conn
merging1 = np.append(emb, emb, axis=1)
merging2 = np.append(merging1, merging1, axis=1)
emb = np.append(merging2, merging2, axis=1)
tmp = nib.load('template_cerebralcortex_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('template_cerebralcortex_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('lenticular_grad1_ProjectedToCerebralCortex_AbsoluteValues.dscalar.nii')

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/home/xavierguell/.local/lib/python3.5/site-packages/nibabel/nifti1.py:582: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  ext_def = np.fromstring(ext_def, dtype=np.int32)


(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
